# Business Understanding

**Problema**: ¿Cómo podemos identificar a los clientes con mayor riesgo de abandono para intervenir de forma proactiva y reducir la tasa de cancelación?
**Hipotesis**: 
- Los clientes con contratos mensuales tienen mayor probabilidad de abandonar que aquellos con contratos a largo plazo.

- El uso de métodos de pago electrónicos automáticos se asocia a una menor tasa de cancelación.

- Clientes con múltiples servicios (internet + teléfono) tienen menor tasa de abandono debido a una mayor dependencia del proveedor.

- Los clientes mayores de 60 años tienen una menor propensión a cancelar su suscripción.

- Una factura mensual más elevada puede estar asociada a una mayor probabilidad de cancelación, especialmente si no hay valor percibido adicional.

**KPI**:
- F1-Score: balance entre precisión y recall.

- AUC-ROC: capacidad del modelo para discriminar entre clases.

Negocio:
- Reducción de la tasa de cancelación: diferencia porcentual antes y después de aplicar las acciones basadas en el modelo.

- Retorno de la inversión (ROI) de promociones: relación entre el costo de los códigos promocionales entregados y los ingresos conservados.

- Tasa de retención de clientes en riesgo: porcentaje de clientes en riesgo que permanecen tras recibir una oferta personalizada.

# Data Understanding

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
contract = pd.read_csv('../data/contract.csv')
internet = pd.read_csv('../data/internet.csv')
personal = pd.read_csv('../data/personal.csv')
phone = pd.read_csv('../data/phone.csv')

In [62]:
def exploracion_inicial(df):
    # Head
    print('-'*30 + 'Primeras 10 filas' + '-'*30)
    print(df.head(10))
    # Estructura de la tabla
    print('-'*30 + 'Estructura de la tabla' + '-'*30)
    print(df.info())
    # Duplicados
    print('-'*30 + 'Duplicados' + '-'*30)
    print(df.duplicated().sum())
    # Valores nulos
    print('-'*30 + 'Valores nulos' + '-'*30)
    print(df.isnull().sum())
    # Estadísticas
    print('-'*30 + 'Estadísticas' + '-'*30)
    print(df.describe())

In [63]:
exploracion_inicial(contract)

------------------------------Primeras 10 filas------------------------------
   customerID   BeginDate              EndDate            Type  \
0  7590-VHVEG  2020-01-01                   No  Month-to-month   
1  5575-GNVDE  2017-04-01                   No        One year   
2  3668-QPYBK  2019-10-01  2019-12-01 00:00:00  Month-to-month   
3  7795-CFOCW  2016-05-01                   No        One year   
4  9237-HQITU  2019-09-01  2019-11-01 00:00:00  Month-to-month   
5  9305-CDSKC  2019-03-01  2019-11-01 00:00:00  Month-to-month   
6  1452-KIOVK  2018-04-01                   No  Month-to-month   
7  6713-OKOMC  2019-04-01                   No  Month-to-month   
8  7892-POOKP  2017-07-01  2019-11-01 00:00:00  Month-to-month   
9  6388-TABGU  2014-12-01                   No        One year   

  PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  
0              Yes           Electronic check           29.85        29.85  
1               No               Mailed c

In [64]:
exploracion_inicial(internet)

------------------------------Primeras 10 filas------------------------------
   customerID InternetService OnlineSecurity OnlineBackup DeviceProtection  \
0  7590-VHVEG             DSL             No          Yes               No   
1  5575-GNVDE             DSL            Yes           No              Yes   
2  3668-QPYBK             DSL            Yes          Yes               No   
3  7795-CFOCW             DSL            Yes           No              Yes   
4  9237-HQITU     Fiber optic             No           No               No   
5  9305-CDSKC     Fiber optic             No           No              Yes   
6  1452-KIOVK     Fiber optic             No          Yes               No   
7  6713-OKOMC             DSL            Yes           No               No   
8  7892-POOKP     Fiber optic             No           No              Yes   
9  6388-TABGU             DSL            Yes          Yes               No   

  TechSupport StreamingTV StreamingMovies  
0          No      

In [65]:
exploracion_inicial(personal)

------------------------------Primeras 10 filas------------------------------
   customerID  gender  SeniorCitizen Partner Dependents
0  7590-VHVEG  Female              0     Yes         No
1  5575-GNVDE    Male              0      No         No
2  3668-QPYBK    Male              0      No         No
3  7795-CFOCW    Male              0      No         No
4  9237-HQITU  Female              0      No         No
5  9305-CDSKC  Female              0      No         No
6  1452-KIOVK    Male              0      No        Yes
7  6713-OKOMC  Female              0      No         No
8  7892-POOKP  Female              0     Yes         No
9  6388-TABGU    Male              0      No        Yes
------------------------------Estructura de la tabla------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-nu

In [66]:
exploracion_inicial(phone)

------------------------------Primeras 10 filas------------------------------
   customerID MultipleLines
0  5575-GNVDE            No
1  3668-QPYBK            No
2  9237-HQITU            No
3  9305-CDSKC           Yes
4  1452-KIOVK           Yes
5  7892-POOKP           Yes
6  6388-TABGU            No
7  9763-GRSKD            No
8  7469-LKBCI            No
9  8091-TTVAX           Yes
------------------------------Estructura de la tabla------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
None
------------------------------Duplicados------------------------------
0
------------------------------Valores nulos------------------------------
customerID       0
MultipleLines    0
dtype: int64
---------

# Data Preparation

In [67]:
# Unificación de fuentes
data = pd.merge(contract, internet, on='customerID', how='left')
data.columns

Index(['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies'],
      dtype='object')

In [68]:
data = pd.merge(data, personal, on='customerID', how='left')
data.columns

Index(['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents'],
      dtype='object')

In [69]:
data = pd.merge(data, phone, on='customerID', how='left')
data.columns

Index(['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'MultipleLines'],
      dtype='object')

In [70]:
exploracion_inicial(data)

------------------------------Primeras 10 filas------------------------------
   customerID   BeginDate              EndDate            Type  \
0  7590-VHVEG  2020-01-01                   No  Month-to-month   
1  5575-GNVDE  2017-04-01                   No        One year   
2  3668-QPYBK  2019-10-01  2019-12-01 00:00:00  Month-to-month   
3  7795-CFOCW  2016-05-01                   No        One year   
4  9237-HQITU  2019-09-01  2019-11-01 00:00:00  Month-to-month   
5  9305-CDSKC  2019-03-01  2019-11-01 00:00:00  Month-to-month   
6  1452-KIOVK  2018-04-01                   No  Month-to-month   
7  6713-OKOMC  2019-04-01                   No  Month-to-month   
8  7892-POOKP  2017-07-01  2019-11-01 00:00:00  Month-to-month   
9  6388-TABGU  2014-12-01                   No        One year   

  PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  \
0              Yes           Electronic check           29.85        29.85   
1               No               Mailed

In [71]:
cols_servicios = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 
                  'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for col in cols_servicios:
    data[col] = data[col].fillna('No')

In [72]:
data['MultipleLines'] = data['MultipleLines'].fillna('No phone service')

In [73]:
# Ver nulos
print(data.isnull().sum())

customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
MultipleLines       0
dtype: int64


In [74]:
# estructura de la tabla
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  gender            7043 non-null   object 
 16  SeniorCitizen     7043 non-null   int64  


In [75]:
# Pasar a fecha BeginDate y EndDate
data['BeginDate'] = pd.to_datetime(data['BeginDate'], format='%Y-%m-%d')

In [76]:
# Para EndDate si no es no pasar a fecha
data['EndDate'] = pd.to_datetime(data['EndDate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [77]:
data['EndDate'].isnull().sum()

5174

In [78]:
data_ml = data.copy()

In [79]:
data_ml['Type'].value_counts()

Type
Month-to-month    3875
Two year          1695
One year          1473
Name: count, dtype: int64

In [80]:
from sklearn.preprocessing import OrdinalEncoder

type_order = [['Month-to-month', 'One year', 'Two year']]
encoder = OrdinalEncoder(categories=type_order)

data_ml['Type_encoded'] = encoder.fit_transform(data_ml[['Type']])
data_ml.drop('Type', axis=1, inplace=True)

In [81]:
data_ml['PaperlessBilling'] = data_ml['PaperlessBilling'].map({'Yes': 1, 'No': 0})

In [82]:
data_ml['PaymentMethod'].value_counts()

PaymentMethod
Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: count, dtype: int64

In [83]:
payment_dummy = pd.get_dummies(data_ml['PaymentMethod'], prefix='PaymentMethod')
data_ml = pd.concat([data_ml, payment_dummy], axis=1)
data_ml.drop('PaymentMethod', axis=1, inplace=True)

In [84]:
data_ml['TotalCharges'] = pd.to_numeric(data_ml['TotalCharges'], errors='coerce')
data_ml['TotalCharges'] = data_ml['TotalCharges'].fillna(0)


In [85]:
# Dummies de InternetService
data_ml = pd.get_dummies(data_ml, columns=['InternetService'], drop_first=False)

In [86]:
data_ml.drop('InternetService_No', axis=1, inplace=True)

In [87]:
# Transforma a boolean OnlineSecurity
data_ml['OnlineSecurity'] = data_ml['OnlineSecurity'].map({'Yes': 1, 'No': 0})

In [88]:
data_ml['OnlineBackup'] = data_ml['OnlineBackup'].map({'Yes': 1, 'No': 0})

In [89]:
data_ml['DeviceProtection'] = data_ml['DeviceProtection'].map({'Yes': 1, 'No': 0})

In [90]:
data_ml['TechSupport'] = data_ml['TechSupport'].map({'Yes': 1, 'No': 0})

In [91]:
data_ml['StreamingTV'] = data_ml['StreamingTV'].map({'Yes': 1, 'No': 0})

In [92]:
data_ml['StreamingMovies'] = data_ml['StreamingMovies'].map({'Yes': 1, 'No': 0})

In [93]:
data_ml['gender'].value_counts()

gender
Male      3555
Female    3488
Name: count, dtype: int64

In [94]:
# Si gender Male 0 si gender Female 1
data_ml['gender'] = data_ml['gender'].map({'Male': 0, 'Female': 1})

In [95]:
data_ml['Partner'] = data_ml['Partner'].map({'Yes': 1, 'No': 0})

In [96]:
data_ml['Dependents'] = data_ml['Dependents'].map({'Yes': 1, 'No': 0})

In [97]:
data_ml = pd.get_dummies(data_ml, columns=['MultipleLines'], drop_first=False)

In [98]:
data_ml.drop('MultipleLines_No phone service', axis=1, inplace=True)

In [99]:
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customerID                               7043 non-null   object        
 1   BeginDate                                7043 non-null   datetime64[ns]
 2   EndDate                                  1869 non-null   datetime64[ns]
 3   PaperlessBilling                         7043 non-null   int64         
 4   MonthlyCharges                           7043 non-null   float64       
 5   TotalCharges                             7043 non-null   float64       
 6   OnlineSecurity                           7043 non-null   int64         
 7   OnlineBackup                             7043 non-null   int64         
 8   DeviceProtection                         7043 non-null   int64         
 9   TechSupport                              

In [100]:
data_ml['is_churn'] = data_ml['EndDate'].apply(lambda x: 1 if pd.notnull(x) else 0)


In [101]:
data_ml['is_churn'].value_counts()

is_churn
0    5174
1    1869
Name: count, dtype: int64

In [102]:
# Duration 
data_ml['Duration'] = (data_ml['EndDate'] - data_ml['BeginDate']).dt.days / 30

In [103]:
data_ml['Duration'].describe()  

count    1869.000000
mean       18.245069
std        19.812987
min         1.000000
25%         2.033333
50%        10.133333
75%        29.433333
max        73.033333
Name: Duration, dtype: float64

In [104]:
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customerID                               7043 non-null   object        
 1   BeginDate                                7043 non-null   datetime64[ns]
 2   EndDate                                  1869 non-null   datetime64[ns]
 3   PaperlessBilling                         7043 non-null   int64         
 4   MonthlyCharges                           7043 non-null   float64       
 5   TotalCharges                             7043 non-null   float64       
 6   OnlineSecurity                           7043 non-null   int64         
 7   OnlineBackup                             7043 non-null   int64         
 8   DeviceProtection                         7043 non-null   int64         
 9   TechSupport                              

In [105]:
# LLena los nulos con un valor alto 
data_ml['Duration'] = data_ml['Duration'].fillna(1000)

In [106]:
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customerID                               7043 non-null   object        
 1   BeginDate                                7043 non-null   datetime64[ns]
 2   EndDate                                  1869 non-null   datetime64[ns]
 3   PaperlessBilling                         7043 non-null   int64         
 4   MonthlyCharges                           7043 non-null   float64       
 5   TotalCharges                             7043 non-null   float64       
 6   OnlineSecurity                           7043 non-null   int64         
 7   OnlineBackup                             7043 non-null   int64         
 8   DeviceProtection                         7043 non-null   int64         
 9   TechSupport                              

In [107]:
exploracion_inicial(data_ml)

------------------------------Primeras 10 filas------------------------------
   customerID  BeginDate    EndDate  PaperlessBilling  MonthlyCharges  \
0  7590-VHVEG 2020-01-01        NaT                 1           29.85   
1  5575-GNVDE 2017-04-01        NaT                 0           56.95   
2  3668-QPYBK 2019-10-01 2019-12-01                 1           53.85   
3  7795-CFOCW 2016-05-01        NaT                 0           42.30   
4  9237-HQITU 2019-09-01 2019-11-01                 1           70.70   
5  9305-CDSKC 2019-03-01 2019-11-01                 1           99.65   
6  1452-KIOVK 2018-04-01        NaT                 1           89.10   
7  6713-OKOMC 2019-04-01        NaT                 0           29.75   
8  7892-POOKP 2017-07-01 2019-11-01                 1          104.80   
9  6388-TABGU 2014-12-01        NaT                 0           56.15   

   TotalCharges  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  \
0         29.85               0       

In [108]:
data_ml.columns = [col.lower().replace(' ', '_') for col in data_ml.columns]

In [109]:
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   customerid                               7043 non-null   object        
 1   begindate                                7043 non-null   datetime64[ns]
 2   enddate                                  1869 non-null   datetime64[ns]
 3   paperlessbilling                         7043 non-null   int64         
 4   monthlycharges                           7043 non-null   float64       
 5   totalcharges                             7043 non-null   float64       
 6   onlinesecurity                           7043 non-null   int64         
 7   onlinebackup                             7043 non-null   int64         
 8   deviceprotection                         7043 non-null   int64         
 9   techsupport                              

In [110]:
# Saca porcentaje de is_churn
data_ml['is_churn'].value_counts(normalize=True)

is_churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

In [114]:
df_ml = data_ml.drop(['customerid', 'begindate', 'enddate'], axis=1)

In [118]:
df_ml['duration'].describe()

count    7043.000000
mean      739.471821
std       433.625076
min         1.000000
25%        58.900000
50%      1000.000000
75%      1000.000000
max      1000.000000
Name: duration, dtype: float64

In [116]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   paperlessbilling                         7043 non-null   int64  
 1   monthlycharges                           7043 non-null   float64
 2   totalcharges                             7043 non-null   float64
 3   onlinesecurity                           7043 non-null   int64  
 4   onlinebackup                             7043 non-null   int64  
 5   deviceprotection                         7043 non-null   int64  
 6   techsupport                              7043 non-null   int64  
 7   streamingtv                              7043 non-null   int64  
 8   streamingmovies                          7043 non-null   int64  
 9   gender                                   7043 non-null   int64  
 10  seniorcitizen                            7043 no

In [117]:
# Train test split
from sklearn.model_selection import train_test_split
X = df_ml.drop('is_churn', axis=1)
y = df_ml['is_churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modeling

# Evaluation

# Deployment

In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [113]:
# 